In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall nltk -y
!pip install texthero

Uninstalling nltk-3.2.5:
  Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 245kB 7.2MB/s 
     |████████████████████████████████| 1.4MB 11.2MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434675 sha256=d76bc5520213b81f1ba9bd3d9b5ca11210297e728941e810b5246b9d4ac1096d
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [ ]:
import os
import numpy as np
import pandas as pd
import gc
from nltk.corpus import stopwords
import nltk
import texthero as hero
from texthero import preprocessing

INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

データの読み込み

In [ ]:
train_df = pd.read_json(os.path.join(INPUT_DIR, 'train_data.json'), lines=True)
test_df = pd.read_json(os.path.join(INPUT_DIR, 'test_data.json'), lines=True)
all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
print(train_df.shape, test_df.shape, all_df.shape)

train_df.head()

(851524, 16) (59084, 15) (910608, 16)


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[{'version': 'v1', 'created': 'Wed, 10 Feb 199...",2008-11-26,"[[Hayes, Mark E., , University College London]...",1,NaN
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[{'version': 'v1', 'created': 'Thu, 27 Mar 201...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[{'version': 'v1', 'created': 'Thu, 22 May 201...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0
3,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[{'version': 'v1', 'created': 'Tue, 3 Jul 2018...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",6,8.0
4,1905.05921,Juanjuan Gu,Juanjuan Gu and Yun Jing,A Modified Mixed Domain Method for Modeling Ac...,None,None,10.1121/10.0001454,None,physics.med-ph physics.comp-ph,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, phase correction and amplitud...","[{'version': 'v1', 'created': 'Wed, 15 May 201...",2020-07-15,"[[Gu, Juanjuan, ], [Jing, Yun, ]]",0,NaN


In [ ]:
all_df.isnull().sum()

id                     0
submitter           9880
authors                0
title                  0
comments          115142
journal-ref       317327
doi                    0
report-no         796898
categories             0
license           290654
abstract               0
versions               0
update_date            0
authors_parsed         0
doi_cites              0
cites             895491
dtype: int64

In [ ]:
del train_df
del test_df
gc.collect()

106

In [ ]:
# カラムを絞る
all_title_abstract = all_df[['title', 'abstract', 'cites']]

#create a custom cleaning pipeline
custom_pipeline = [preprocessing.fillna
                   ,preprocessing.lowercase
                   ,preprocessing.remove_digits
                   ,preprocessing.remove_punctuation
                   ,preprocessing.remove_diacritics
                   ,preprocessing.remove_whitespace
                   ,preprocessing.remove_stopwords
                  ]

all_title_abstract['clean_title'] = hero.clean(all_title_abstract['title'], 
                                               pipeline=custom_pipeline).values
all_title_abstract['clean_abstract'] = hero.clean(all_title_abstract['abstract'], 
                                                  pipeline=custom_pipeline).values
all_title_abstract.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,title,abstract,cites,clean_title,clean_abstract
0,Heavy-Flavour Production at HERA,We review the theoretical and experimental s...,NaN,heavy flavour production hera,review theoretical experimental status hea...
1,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
2,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
3,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
4,A Modified Mixed Domain Method for Modeling Ac...,"In this paper, phase correction and amplitud...",NaN,modified mixed domain method modeling acoust...,paper phase correction amplitude compensati...
5,Gaussian asymptotic limits for the $\alpha$-tr...,Compositional data consists of vectors of pr...,NaN,gaussian asymptotic limits \alpha transforma...,compositional data consists vectors proporti...
6,Early X-ray/UV Line Signatures of GRB Progenit...,We calculate the X-ray/UV spectral line sign...,2.0,early x ray uv line signatures grb progenitor...,calculate x ray uv spectral line signatures ...
7,Neutrino oscillations with three flavors in ma...,"In this paper, we discuss the evolution oper...",14.0,neutrino oscillations three flavors matter ...,paper discuss evolution operator transit...
8,Monte Carlo method for polarized radiative tra...,Light transfer in gradient-index media gener...,16.0,monte carlo method polarized radiative transf...,light transfer gradient index media generally...
9,A Framework for Designing and Evaluating Solar...,Disturbances in space weather can negatively...,NaN,framework designing evaluating solar flare ...,disturbances space weather negatively affect...


In [ ]:
del all_df
gc.collect()

82

In [ ]:
all_title_abstract.to_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))